# Simulation

This simulation shows how the app should receive the accident information, and what will be the result

In [ ]:
# libraries 
import pandas as pd
import geopandas as gpd
import folium 

# Custom Libraries
#import project.source.locator as loc
#import project.source.mapquest as mq 

: 

## Receiving an accident alert

In [2]:
# This code block instances locator class to create Lat-Lon point and its buffer.
# The buffer will be used to determine what agents are capable to attend the call.
accident = dict()

# The operator should type the address. The direction could be a place or the coordinates
# In this case, we'll use a place name:
accident['address'] = 'Parque el Virrey, Bogotá'

# To determine the priority is necessary to know if any dead or injured
accident['dead'] = 0
accident['injured'] = 1

# TODO: This radius should be calculated based on history
accident['radius'] = 2000

accident = gpd.GeoDataFrame([accident])

In [3]:
# The map with the accident location is created
locate = loc.Locator()
location = locate.get_location(accident['address'][0])
if location:
    accident = locate.make_accident_point(accident)
    buffer = locate.make_buffer(accident)

# accident store all the information
accident

,address,dead,injured,radius,geometry
0,"Parque El Virrey, UPZ Chicó Lago, Localidad Ch...",0,1,2000,POINT (-74.05208 4.67213)


In [4]:
# Viewing the above information
# The marker is the accident location and the buffer is the circle around it.

# Create the map centered in the accident location
m = folium.Map((location.latitude, location.longitude), max_zoom=20, zoom_start=14)

# Add the buffer layer to the map
folium.GeoJson(buffer).add_to(m)

# Add the accident market to the map
folium.Marker(
    location=[accident.geometry[0].y, accident.geometry[0].x],
    tooltip=accident.address[0],
    icon=folium.Icon(color='darkblue', icon='star'),
).add_to(m)

# Shows the map
m

## Checking Agents

In [5]:
# Creating N random agents in the city
agents = locate.make_agents(50)
agents.head()

,geometry,address,agent_idx
0,POINT (-74.21531 4.45916),"Vía El Cardal - Romeral, Hungría, Soacha, Hung...",0
1,POINT (-74.04482 4.52439),"El Delirio - Páramo de Cruz Verde, Localidad S...",1
2,POINT (-74.16343 4.74973),"Vía Funza - Siberia, Siberia, La Florida, Cota...",2
3,POINT (-74.20759 4.48105),"Hungría, Soacha, Hungría, Corregimiento 1 Sur,...",3
4,POINT (-73.97005 4.50076),"Ubaque, Oriente, Cundinamarca, Colombia",4


In [6]:
# Add the agents to the previous map
for geometry, address, agent_idx in agents.itertuples(index=False):
    folium.Marker(location=[geometry.y, geometry.x],
                  tooltip=f'Agent: {agent_idx}',
                  icon=folium.Icon(color='darkgreen', icon='user')).add_to(m)

#Shows the new map
m

In [7]:
# Get the agents that are within the buffer area
potential_agents = locate.potential_agents(agents, buffer)
potential_agents

,geometry,address,agent_idx
0,POINT (-74.06029 4.67509),"Carrera 21, Polo Club, UPZ Los Alcázares, Loca...",37


# Calculate the time and distance for the potential agents

In [10]:
mapquest = mq.MapQuest()

# route method receives two values: 
# location1: this value will be the agent coordinates (lat,lon)
# location2: this value will be the accident coordinates (lat,lon)

for idx in potential_agents.index:
    directions = (mapquest.route(
        f"'{potential_agents.geometry[idx].y},{potential_agents.geometry[idx].x}'",
        f"'{accident.geometry[0].y},{accident.geometry[0].x}'"))

In [15]:
directions.keys()

dict_keys(['route', 'info'])

In [17]:
directions['route']

{'hasTollRoad': True,
 'hasBridge': True,
 'boundingBox': {'lr': {'lng': -96.264297, 'lat': 36.253082},
  'ul': {'lng': -99.064087, 'lat': 39.457764}},
 'distance': 370.033,
 'hasTimedRestriction': False,
 'hasTunnel': False,
 'hasHighway': True,
 'computedWaypoints': [],
 'routeError': {'errorCode': -400, 'message': ''},
 'formattedTime': '06:00:46',
 'sessionId': '62a6a155-0183-6750-02b4-3708-0eea4e076caf',
 'hasAccessRestriction': False,
 'realTime': 22380,
 'hasSeasonalClosure': False,
 'hasCountryCross': False,
 'fuelUsed': 18.47,
 'legs': [{'hasTollRoad': True,
   'hasBridge': True,
   'destNarrative': 'Proceed to US.',
   'distance': 370.033,
   'hasTimedRestriction': False,
   'hasTunnel': False,
   'hasHighway': True,
   'index': 0,
   'formattedTime': '06:00:46',
   'origIndex': 1,
   'hasAccessRestriction': False,
   'hasSeasonalClosure': False,
   'hasCountryCross': False,
   'roadGradeStrategy': [[]],
   'destIndex': 16,
   'time': 21646,
   'hasUnpaved': True,
   'origNar

In [48]:
directions_df = pd.DataFrame([directions['route']])
directions_df['legs'].keys()

RangeIndex(start=0, stop=1, step=1)

In [53]:
pd.DataFrame(directions_df['legs'].iloc[0])['maneuvers']

0    [{'distance': 0.995, 'streets': ['Lake Dr'], '...
Name: maneuvers, dtype: object